In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install SimpleITK
!pip install pyradiomics

In [6]:
import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

# Initialize PyRadiomics extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

# Update settings to enable angle-wise extraction and disable aggregation
extractor.settings['force2D'] = True  # Ensure features are extracted for each angle in 2D
extractor.settings['enableAngleAverage'] = False  # Disable angle aggregation to extract angle-specific features
extractor.settings['binWidth'] = 25  # Example bin width (can be adjusted as needed)
extractor.settings['resampledPixelSpacing'] = None  # Use original spacing
extractor.settings['interpolator'] = 'sitkBSpline'  # Interpolation method
extractor.settings['normalize'] = True  # Normalize images before feature extraction

# Paths to image and mask folders
image_folder = "/content/drive/MyDrive/FYP/Dataset/Covid"  # Replace with your image folder path
mask_folder = "/content/drive/MyDrive/FYP/Dataset/Covid_Masks"  # Replace with your mask folder path

# Prepare the output CSV file
output_csv = "/content/drive/MyDrive/FYP/extracted_features_angle_wise_Covid.csv"
feature_data = []

# Get all image and mask files (assuming they are paired and ordered in the folders)
image_files = sorted(os.listdir(image_folder))
mask_files = sorted(os.listdir(mask_folder))

# Define the angles for angle-wise feature extraction
angles = [0, 45, 90, 135]

# Loop through each image and corresponding mask
for img_file, mask_file in zip(image_files, mask_files):
    image_path = os.path.join(image_folder, img_file)
    mask_path = os.path.join(mask_folder, mask_file)

    # Read the image and mask
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)

    # Convert images to grayscale if necessary
    if image.GetNumberOfComponentsPerPixel() > 1:
        image = sitk.VectorIndexSelectionCast(image, 0)  # Select one channel, e.g., Red channel

    if mask.GetNumberOfComponentsPerPixel() > 1:
        mask = sitk.VectorIndexSelectionCast(mask, 0)  # Ensure mask is single-channel as well

    # Ensure images are of type UInt8 or UInt16, compatible with radiomics processing
    image = sitk.Cast(image, sitk.sitkUInt8)
    mask = sitk.Cast(mask, sitk.sitkUInt8)

    # Check if the image and mask sizes match
    if image.GetSize() != mask.GetSize():
        print(f"Resizing mask for {img_file} to match image dimensions.")
        mask = sitk.Resample(mask, referenceImage=image, transform=sitk.Transform(),
                             interpolator=sitk.sitkNearestNeighbor, defaultPixelValue=0, outputPixelType=sitk.sitkUInt8)

    # Extract all radiomic features angle-wise
    print(f"Extracting features for {img_file}...")
    features = extractor.execute(image, mask)

    # Store features in a dictionary, with image filename as identifier
    feature_row = {"Image": img_file}

    # Loop through the features and add angle-specific features to the row
    for feature_name, feature_value in features.items():
        if "_angle" in feature_name:
            # Handle angle-specific features
            for angle in angles:
                if f"_angle_{angle}" in feature_name:
                    feature_row[f"{feature_name}_angle_{angle}"] = feature_value
        else:
            # For non-angle features, save them directly
            feature_row[feature_name] = feature_value

    # Append the row to the data list
    feature_data.append(feature_row)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(feature_data)

# Save the DataFrame to CSV
df.to_csv(output_csv, index=False)
print(f"Angle-wise feature extraction complete. Results saved to {output_csv}")


Output hidden; open in https://colab.research.google.com to view.